- remove outlier patient
- implement transfer learning correctly in cpu

In [1]:
import os
import sys
import random
import warnings
import numpy as np
import pandas as pd 
from sklearn import preprocessing
from sklearn.metrics import log_loss, mean_squared_error
from sklearn.decomposition import PCA
from tqdm import tqdm_notebook as tqdm
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_selection import VarianceThreshold

sys.path.append('../input/multilabelstraifier/')
sys.path.append('../input/lookahead/')
from ml_stratifiers import MultilabelStratifiedKFold
from lookahead import Lookahead
warnings.filterwarnings('ignore')

import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf

In [2]:
DATA_DIR = '/kaggle/input/lish-moa/'
train = pd.read_csv(DATA_DIR + 'train_features.csv')
targets = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
non_targets = pd.read_csv(DATA_DIR + 'train_targets_nonscored.csv')
test = pd.read_csv(DATA_DIR + 'test_features.csv')
sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [3]:
target_feats = [ i for i in targets.columns if i != "sig_id"]
g_feats = [i for i in train.columns if "g-" in i]
c_feats = [i for i in train.columns if "c-" in i]

In [4]:
# to remove outlier paienr
original_remove_index = [16674] # remove_index = [15361]

In [5]:
train.loc[original_remove_index,:]

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
16674,id_b31edc707,trt_cp,72,D2,1.902,3.796,3.323,10.0,2.016,-2.356,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-9.28,-9.719,-6.003


In [6]:
noncons_train_index = train[train.cp_type=="ctl_vehicle"].index
noncons_train_index = list(noncons_train_index) + original_remove_index
cons_train_index = train[~train.index.isin(noncons_train_index)].index
noncons_test_index = test[test.cp_type=="ctl_vehicle"].index
cons_test_index = test[test.cp_type!="ctl_vehicle"].index

# preprocess

In [7]:
train = train[train.index.isin(cons_train_index)].copy().reset_index(drop=True)
targets = targets[targets.index.isin(cons_train_index)].copy().reset_index(drop=True)
non_targets = non_targets[non_targets.index.isin(cons_train_index)].copy().reset_index(drop=True)

In [8]:
non_target_feats = [i for i in non_targets.columns if i != "sig_id"]
nontarget_dists = pd.DataFrame(np.sum(non_targets[non_target_feats])).reset_index(drop=False)
nontarget_dists.columns = ["target", "number"]
nontarget_dists = nontarget_dists.sort_values("number", ascending=False).reset_index(drop=True)
drop_list1 = list(nontarget_dists[nontarget_dists.number==0]["target"].values)
print("first drop", len(drop_list1))
non_targets.drop(drop_list1, axis=1, inplace=True)
print("shape after 1st drop:", non_targets.shape)
non_target_feats = [i for i in non_targets.columns if i != "sig_id"]
print(len(non_target_feats))

first drop 71
shape after 1st drop: (21947, 332)
331


In [9]:
all_targets = pd.concat([targets, non_targets], axis=1)

# Feature engineering 

In [10]:
#num = 10
#pca_c_cols = ["pca-c"+str(i+1) for i in range(num)]
#pca = PCA(n_components=num)
#tmp_train = pca.fit_transform(train[c_feats])
#tmp_test = pca.transform(test[c_feats])
#tmp_train = pd.DataFrame(tmp_train, columns=pca_c_cols)
#tmp_test = pd.DataFrame(tmp_test, columns=pca_c_cols)

#train = pd.concat([train, tmp_train],axis=1)
#test = pd.concat([test, tmp_test],axis=1)

In [11]:
def fe(df):
    tmp = df.copy()
    #tmp['g_sum'] = tmp[g_feats].sum(axis = 1)
    #tmp['g_mean'] = tmp[g_feats].mean(axis = 1)
    #tmp['g_std'] = tmp[g_feats].std(axis = 1)
    #tmp['g_kurt'] = tmp[g_feats].kurtosis(axis = 1)
    #tmp['g_skew'] = tmp[g_feats].skew(axis = 1)
    #tmp['c_sum'] = tmp[c_feats].sum(axis = 1)
    #tmp['c_mean'] = tmp[c_feats].mean(axis = 1)
    #tmp['c_std'] = tmp[c_feats].std(axis = 1)
    #tmp['c_kurt'] = tmp[c_feats].kurtosis(axis = 1)
    #tmp['c_skew'] = tmp[c_feats].skew(axis = 1)
    #tmp['gc_sum'] = tmp[c_feats + g_feats].sum(axis = 1)
    #tmp['gc_mean'] = tmp[c_feats + g_feats].mean(axis = 1)
    #tmp['gc_std'] = tmp[c_feats + g_feats].std(axis = 1)
    #tmp['gc_kurt'] = tmp[c_feats + g_feats].kurtosis(axis = 1)
    #tmp['gc_skew'] = tmp[c_feats + g_feats].skew(axis = 1)
    #tmp.loc[:, 'cp_type'] = tmp.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    #tmp.loc[:, 'cp_dose'] = tmp.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
        
    tmp.drop(["cp_type", "sig_id", "cp_time", "cp_dose"], axis=1, inplace=True)
    return tmp

f_train = fe(train)
f_test = fe(test)

print(f_train.shape, f_test.shape)

(21947, 872) (3982, 872)


In [12]:
fn_train = f_train.copy().to_numpy()
fn_test = f_test.copy().to_numpy()

ss = preprocessing.RobustScaler()
fn_train= ss.fit_transform(fn_train)
fn_test = ss.transform(fn_test)

#fn_nontargets = non_targets.drop("sig_id", axis=1).copy().to_numpy()
fn_targets = targets.drop("sig_id", axis=1).copy().to_numpy()
fn_all_targets = all_targets.drop("sig_id", axis=1).copy().to_numpy()

# modelling

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
def mean_log_loss(y_true, y_pred):
    metrics = []
    for i, target in enumerate(target_feats):
        metrics.append(log_loss(y_true[:, i], y_pred[:, i].astype(float), labels=[0,1]))
    return np.mean(metrics)

def seed_everything(seed=1234): 
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

class MoaModel(nn.Module):
    def __init__(self, num_columns, last_num):
        super(MoaModel, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_columns)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_columns, 2048))
        
        self.batch_norm2 = nn.BatchNorm1d(2048)
        self.dropout2 = nn.Dropout(0.6)
        self.dense2 = nn.utils.weight_norm(nn.Linear(2048,1048))
        
        self.batch_norm3 = nn.BatchNorm1d(1048)
        self.dropout3 = nn.Dropout(0.6)
        self.dense3 = nn.utils.weight_norm(nn.Linear(1048, last_num))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

cpu


# modelling

In [14]:
batch_size = 128
n_folds=5
EARLY_STOPPING_STEPS = 10

def modelling_torch(tr, target, te, sample_seed, init_num, last_num, files):
    seed_everything(seed=sample_seed) 
    X_train = tr.copy()
    y_train = target.copy()
    X_test = te.copy()
    test_len = X_test.shape[0]
    
    if files == []:
        train_epochs = 30
        mskf=MultilabelStratifiedKFold(n_splits = n_folds, shuffle=True, random_state=224)
        ans = []
    else:
        train_epochs = 10
        mskf=MultilabelStratifiedKFold(n_splits = n_folds, shuffle=True, random_state=2)

    models = []
    
    X_test = torch.tensor(X_test, dtype=torch.float32)
    X_test = torch.utils.data.TensorDataset(X_test) 
    test_loader = torch.utils.data.DataLoader(X_test, batch_size=batch_size, shuffle=False)
    
    oof = np.zeros([len(X_train),y_train.shape[1]])
    oof_targets = np.zeros([len(X_train),y_train.shape[1]])
    pred_value = np.zeros([test_len, y_train.shape[1]])
    scores = []
    for fold, (train_index, valid_index) in enumerate(mskf.split(X_train, y_train)):
        print("Fold "+str(fold+1))
        X_train2 = torch.tensor(X_train[train_index,:], dtype=torch.float32)
        y_train2 = torch.tensor(y_train[train_index], dtype=torch.float32)

        X_valid2 = torch.tensor(X_train[valid_index,:], dtype=torch.float32)
        y_valid2 = torch.tensor(y_train[valid_index], dtype=torch.float32)
            
        clf = MoaModel(init_num, len(non_target_feats)+len(target_feats))
        loss_fn = torch.nn.BCEWithLogitsLoss() 
        if files != []:
            clf.load_state_dict(torch.load(files[fold]))
            for param in clf.parameters():
                param.requires_grad = False
            # reinitialze
            clf.dense3 = nn.utils.weight_norm(nn.Linear(1048, len(target_feats)))
            for param in clf.dense3.parameters():
                param.requires_grad = True
            optimizer = optim.Adam(clf.parameters(), lr = 0.001, weight_decay=1e-5) 
            #lookahead = Lookahead(optimizer, k=10, alpha=0.6) #lookahead
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, eps=1e-4, verbose=True)
        else:
            optimizer = optim.Adam(clf.parameters(), lr = 0.001, weight_decay=1e-5) 
            #lookahead = Lookahead(optimizer, k=10, alpha=0.6) #lookahead
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, eps=1e-4, verbose=True)
        
        train = torch.utils.data.TensorDataset(X_train2, y_train2)
        valid = torch.utils.data.TensorDataset(X_valid2, y_valid2)
        
        clf.to(device)
        
        train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True) 
        valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)
        
        best_val_loss = np.inf
        stop_counts = 0
        for epoch in range(train_epochs):
            start_time = time.time()
            clf.train()
            avg_loss = 0.
            for x_batch, y_batch in tqdm(train_loader, disable=True):
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)
                y_pred = clf(x_batch) 
                loss = loss_fn(y_pred, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                avg_loss += loss.item() / len(train_loader)        
            
            clf.eval()
            avg_val_loss = 0.
            for i, (x_batch, y_batch) in enumerate(valid_loader): 
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)
                y_pred = clf(x_batch).detach()
                avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
        
            elapsed_time = time.time() - start_time 
            scheduler.step(avg_val_loss)
                    
            if avg_val_loss < best_val_loss:
                stop_counts = 0
                best_val_loss = avg_val_loss
                print('Best model: Epoch {} \t loss={:.6f} \t val_loss={:.6f} \t time={:.2f}s'.format(
                    epoch + 1, avg_loss, avg_val_loss, elapsed_time))
                if files == []:
                    torch.save(clf.state_dict(), 'parameters'+str(fold+1)+'.pt')
                else:    
                    torch.save(clf.state_dict(), 'best-model-parameters.pt')
            else:
                stop_counts += 1
        
            #if stop_counts >= EARLY_STOPPING_STEPS: 
            #    break
            
        if files == []:
            pred_model = MoaModel(init_num, len(non_target_feats)+len(target_feats))
            ans.append('parameters'+str(fold+1)+'.pt')
            pred_model.load_state_dict(torch.load('parameters'+str(fold+1)+'.pt'))
        else:
            pred_model = MoaModel(init_num, len(target_feats))
            pred_model.load_state_dict(torch.load('best-model-parameters.pt'))         
        pred_model.eval()
        
        # validation check ----------------
        oof_epoch = np.zeros([X_valid2.size(0), y_train.shape[1]])
        target_epoch = np.zeros([X_valid2.size(0), y_train.shape[1]])
        for i, (x_batch, y_batch) in enumerate(valid_loader): 
                y_pred = pred_model(x_batch).sigmoid().detach()
                oof_epoch[i * batch_size:(i+1) * batch_size,:] = y_pred.cpu().numpy()
                target_epoch[i * batch_size:(i+1) * batch_size,:] = y_batch.cpu().numpy()
        print("Fold {} log loss: {}".format(fold+1, mean_log_loss(target_epoch, oof_epoch)))
        scores.append(mean_log_loss(target_epoch, oof_epoch))
        oof[valid_index,:] = oof_epoch
        oof_targets[valid_index,:] = target_epoch
        #-----------------------------------
        
        if files != []:
            # test predcition --------------
            test_preds = np.zeros([test_len, y_train.shape[1]])
            for i, (x_batch,) in enumerate(test_loader): 
                y_pred = pred_model(x_batch).sigmoid().detach()
                test_preds[i * batch_size:(i+1) * batch_size, :] = y_pred.cpu().numpy()
            pred_value += test_preds / n_folds
            # ------------------------------
        
    print("Seed {}".format(seed_))
    for i, ele in enumerate(scores):
        print("Fold {} log loss: {}".format(i+1, scores[i]))
    print("Std of log loss: {}".format(np.std(scores)))
    print("Total log loss: {}".format(mean_log_loss(oof_targets, oof)))
    
    if files != []:
        return oof, oof_targets, pred_value
    else:
        return ans

In [15]:
print("Transfer learning")
for seed_ in [0]:
    files = modelling_torch(fn_train, fn_all_targets, fn_test, seed_, fn_train.shape[1], fn_all_targets.shape[1],[])

target_oof = np.zeros([len(fn_train),fn_targets.shape[1]])
target_pred = np.zeros([len(fn_test),fn_targets.shape[1]])

seeds = [10,40]

print("Final prediction")
for seed_ in seeds:
    oof, oof_targets, pytorch_pred = modelling_torch(fn_train, fn_targets, fn_test, seed_, fn_train.shape[1], fn_targets.shape[1], files)
    target_oof += oof / len(seeds)
    target_pred += pytorch_pred / len(seeds)
print("Total log loss in targets: {}".format(mean_log_loss(oof_targets, target_oof)))

Transfer learning
Fold 1
Best model: Epoch 1 	 loss=0.407421 	 val_loss=0.072216 	 time=14.80s
Best model: Epoch 2 	 loss=0.041442 	 val_loss=0.021327 	 time=13.62s
Best model: Epoch 3 	 loss=0.019567 	 val_loss=0.014547 	 time=13.69s
Best model: Epoch 4 	 loss=0.014629 	 val_loss=0.013003 	 time=14.14s
Best model: Epoch 5 	 loss=0.013476 	 val_loss=0.011736 	 time=14.46s
Best model: Epoch 6 	 loss=0.012093 	 val_loss=0.011416 	 time=14.30s
Best model: Epoch 7 	 loss=0.011860 	 val_loss=0.011277 	 time=14.00s
Best model: Epoch 8 	 loss=0.011667 	 val_loss=0.010979 	 time=14.44s
Best model: Epoch 9 	 loss=0.011429 	 val_loss=0.010823 	 time=14.21s
Best model: Epoch 10 	 loss=0.011027 	 val_loss=0.010665 	 time=14.62s
Best model: Epoch 11 	 loss=0.010848 	 val_loss=0.010472 	 time=14.10s
Best model: Epoch 13 	 loss=0.010671 	 val_loss=0.010385 	 time=14.46s
Best model: Epoch 14 	 loss=0.010467 	 val_loss=0.010042 	 time=14.45s
Best model: Epoch 15 	 loss=0.010432 	 val_loss=0.009991 	 ti

In [16]:
t = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
train_checkscore = t.copy()
train_checkscore.loc[train_checkscore.index.isin(cons_train_index),target_feats] = target_oof
train_checkscore.loc[train_checkscore.index.isin(noncons_train_index),target_feats] = 0

train_checkscore.drop(original_remove_index, inplace=True)
t.drop(original_remove_index, inplace=True)
t.drop("sig_id", axis=1, inplace=True)

print('OOF log loss: ', log_loss(np.ravel(t), np.ravel(np.array(train_checkscore.iloc[:,1:]))))

OOF log loss:  0.014392884939624309


In [17]:
sub[target_feats] = target_pred
sub.loc[noncons_test_index,target_feats] = 0
sub.to_csv('submission.csv', index=False)